In [18]:
import pandas as pd
import numpy as np
import pickle as pk
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import f1_score

from keras.models import Sequential, load_model
from keras.layers import Dense, Conv2D, LSTM, CuDNNLSTM, Flatten, Dropout, Reshape, BatchNormalization, PReLU, ELU
from keras import optimizers
from keras import initializers
from keras.callbacks import ReduceLROnPlateau, CSVLogger, ModelCheckpoint
from keras.utils import to_categorical
import keras as ke

In [2]:
def slidingWindow(sequence, labels, winSize, step, noNull):

    # Verify the inputs
    try: it = iter(sequence)
    except TypeError:
        raise Exception("**ERROR** sequence must be iterable.")
    if not ((type(winSize) == type(0)) and (type(step) == type(0))):
        raise Exception("**ERROR** type(winSize) and type(step) must be int.")
    if step > winSize:
        raise Exception("**ERROR** step must not be larger than winSize.")
    if winSize > len(sequence):
        raise Exception("**ERROR** winSize must not be larger than sequence length.")
 
    # number of chunks
    numOfChunks = ((len(sequence)-winSize)//step)+1
 
    # Do the work
    for i in range(0,numOfChunks*step,step):
        segment = sequence[i:i+winSize]
        seg_labels = labels[i:i+winSize]
        if noNull:
            if seg_labels[-1] != 0:
                yield segment, seg_labels
        else:
            yield segment, seg_labels

In [3]:
def segment_data(X_train, y_train, X_val, y_val, X_test, y_test, winSize, step, noNull=False):
    assert len(X_train) == len(y_train)
    assert len(X_val) == len(y_val)
    assert len(X_test) == len(y_test)
    # obtain chunks of data
    train_chunks = slidingWindow(X_train, y_train , winSize, step, noNull)
    val_chunks = slidingWindow(X_val, y_val, winSize, step, noNull)
    test_chunks = slidingWindow(X_test, y_test, winSize, step, noNull)
    
    # segment the data
    train_segments = []
    train_labels = []
    for chunk in train_chunks:
        data = chunk[0]
        labels = chunk[1]
        train_segments.append(data)
        train_labels.append(labels[-1])
        
    val_segments = []
    val_labels = []
    for chunk in val_chunks:
        data = chunk[0]
        labels = chunk[1]
        val_segments.append(data)
        val_labels.append(labels[-1])
    
    test_segments = []
    test_labels = []
    for chunk in test_chunks:
        data = chunk[0]
        labels = chunk[1]
        test_segments.append(data)
        test_labels.append(labels[-1])
        
    return np.array(train_segments), np.array(train_labels), np.array(val_segments), np.array(val_labels), np.array(test_segments), np.array(test_labels)

In [4]:
def prepare_data(train_data, val_data, test_data):
    encoder = OneHotEncoder()
    train_labels = encoder.fit_transform(train_data['labels'].values.reshape(-1,1)).toarray()
    val_labels = encoder.transform(val_data['labels'].values.reshape(-1,1)).toarray()
    test_labels = encoder.transform(test_data['labels'].values.reshape(-1,1)).toarray()
    scaler = MinMaxScaler(feature_range=(0, 1))
    train_data.drop(['labels'], axis=1, inplace=True)
    val_data.drop(['labels'], axis=1, inplace=True)
    test_data.drop(['labels'], axis=1, inplace=True)
    data = pd.concat([train_data,val_data,test_data])
    scaler.fit(data)
    train_data = scaler.transform(train_data)
    val_data = scaler.transform(val_data)
    test_data = scaler.transform(test_data)
    
    return train_data, val_data, test_data, train_labels, val_labels, test_labels

In [6]:
# import train data
adl_1_1 = pd.read_csv("../full_dataset/ADL1Opportunity_taskB2_S1.csv",header=None)
adl_1_2 = pd.read_csv("../full_dataset/ADL2Opportunity_taskB2_S1.csv",header=None)
adl_1_3 = pd.read_csv("../full_dataset/ADL3Opportunity_taskB2_S1.csv",header=None)
adl_1_4 = pd.read_csv("../full_dataset/ADL4Opportunity_taskB2_S1.csv",header=None)
adl_1_5 = pd.read_csv("../full_dataset/ADL5Opportunity_taskB2_S1.csv",header=None)
drill_1 = pd.read_csv("../full_dataset/Drill1Opportunity_taskB2.csv",header=None)
adl_2_1 = pd.read_csv("../full_dataset/ADL1Opportunity_taskB2_S2.csv",header=None)
adl_2_2 = pd.read_csv("../full_dataset/ADL2Opportunity_taskB2_S2.csv",header=None)
drill_2 = pd.read_csv("../full_dataset/Drill2Opportunity_taskB2.csv",header=None)
adl_3_1 = pd.read_csv("../full_dataset/ADL1Opportunity_taskB2_S3.csv",header=None)
adl_3_2 = pd.read_csv("../full_dataset/ADL2Opportunity_taskB2_S3.csv",header=None)
drill_3 = pd.read_csv("../full_dataset/Drill3Opportunity_taskB2.csv",header=None)

# import validation data
adl_2_3 = pd.read_csv("../full_dataset/ADL3Opportunity_taskB2_S2.csv",header=None)
adl_3_3 = pd.read_csv("../full_dataset/ADL3Opportunity_taskB2_S3.csv",header=None)

# import test data
adl_2_4 = pd.read_csv("../full_dataset/ADL4Opportunity_taskB2_S2.csv",header=None)
adl_2_5 = pd.read_csv("../full_dataset/ADL5Opportunity_taskB2_S2.csv",header=None)
adl_3_4 = pd.read_csv("../full_dataset/ADL4Opportunity_taskB2_S3.csv",header=None)
adl_3_5 = pd.read_csv("../full_dataset/ADL5Opportunity_taskB2_S3.csv",header=None)

In [7]:
train_frames = [adl_1_1,adl_1_2,adl_1_3,adl_1_4,adl_1_5,drill_1,adl_2_1,adl_2_2,drill_2,adl_3_1,adl_3_2,drill_3]
val_frames = [adl_2_3,adl_3_3]
test_frames = [adl_2_4,adl_2_5,adl_3_4,adl_3_5]
train_data = pd.concat(train_frames)
val_data = pd.concat(val_frames)
test_data = pd.concat(test_frames)
train_data.rename(columns ={113: 'labels'}, inplace =True)
val_data.rename(columns ={113: 'labels'}, inplace =True)
test_data.rename(columns ={113: 'labels'}, inplace =True)

In [8]:
print("shapes: train {0}, val {1}, test {2}".format(train_data.shape, val_data.shape, test_data.shape))

shapes: train (497014, 114), val (60949, 114), test (118750, 114)


In [9]:
scaled_train, scaled_val, scaled_test, train_labels, val_labels, test_labels = prepare_data(train_data, val_data, test_data)

In [10]:
print(scaled_train.shape)
print(train_labels.shape)

(497014, 113)
(497014, 18)


In [11]:
num_sensors = 113
window_size = 24
step_size = 12
classes = 18

In [12]:
train_segments, train_labels, val_segments, val_labels, test_segments, test_labels = segment_data(scaled_train, train_labels, scaled_val, val_labels,
                                                                                                  scaled_test, test_labels, window_size, step_size)

In [13]:
# reshape input for CNN
reshaped_train = train_segments.reshape(-1, window_size, num_sensors, 1)
reshaped_val = val_segments.reshape(-1, window_size, num_sensors, 1)
reshaped_test = test_segments.reshape(-1, window_size, num_sensors, 1)

In [13]:
model = Sequential()

size_of_kernel = (5,1)
kernel_strides = 1
num_filters = 64
num_lstm_cells = 128
dropout_prob = 0.5
inputshape = (window_size, num_sensors, 1)

In [14]:
model.add(BatchNormalization(input_shape=inputshape))

In [15]:
model.add(Conv2D(num_filters, kernel_size=size_of_kernel, strides=kernel_strides,
                 kernel_initializer='glorot_normal', name='1_conv_layer'))
model.add(PReLU())

In [16]:
model.add(Conv2D(num_filters, kernel_size=size_of_kernel, strides=kernel_strides,
                 kernel_initializer='glorot_normal',
                 name='2_conv_layer'))
model.add(PReLU())

In [17]:
model.add(Conv2D(num_filters, kernel_size=size_of_kernel, strides=kernel_strides,
                 kernel_initializer='glorot_normal',
                 name='3_conv_layer'))
model.add(PReLU())

In [18]:
model.add(Conv2D(num_filters, kernel_size=size_of_kernel, strides=kernel_strides,
                 kernel_initializer='glorot_normal',
                 name='4_conv_layer'))
model.add(PReLU())

In [19]:
model.add(Reshape((8, num_filters*num_sensors)))

In [20]:
model.add(CuDNNLSTM(num_lstm_cells,kernel_initializer='glorot_normal', return_sequences=True, name='1_lstm_layer'))

model.add(Dropout(dropout_prob, name='2_dropout_layer'))

model.add(CuDNNLSTM(num_lstm_cells,kernel_initializer='glorot_normal',return_sequences=False, name='2_lstm_layer'))

model.add(Dropout(dropout_prob, name='3_dropout_layer'))

model.add(Dense(64,kernel_initializer='glorot_normal',
                bias_initializer=initializers.Constant(value=0.1), name='dense_layer'))
model.add(Dropout(dropout_prob, name='4_dropout_layer'))

model.add(Dense(classes,kernel_initializer='glorot_normal',
                bias_initializer=initializers.Constant(value=0.1),activation='softmax', name='softmax_layer'))

opt = optimizers.Adam(lr=0.001)
model.compile(loss='categorical_crossentropy', optimizer=opt, metrics=['accuracy'])

Instructions for updating:
keep_dims is deprecated, use keepdims instead
Instructions for updating:
keep_dims is deprecated, use keepdims instead


In [21]:
for layer in model.layers:
    print(str(layer.name) + ': input shape: ' + str(layer.input_shape) + ' output shape: ' + str(layer.output_shape))

batch_normalization_1: input shape: (None, 24, 113, 1) output shape: (None, 24, 113, 1)
1_conv_layer: input shape: (None, 24, 113, 1) output shape: (None, 20, 113, 64)
p_re_lu_1: input shape: (None, 20, 113, 64) output shape: (None, 20, 113, 64)
2_conv_layer: input shape: (None, 20, 113, 64) output shape: (None, 16, 113, 64)
p_re_lu_2: input shape: (None, 16, 113, 64) output shape: (None, 16, 113, 64)
3_conv_layer: input shape: (None, 16, 113, 64) output shape: (None, 12, 113, 64)
p_re_lu_3: input shape: (None, 12, 113, 64) output shape: (None, 12, 113, 64)
4_conv_layer: input shape: (None, 12, 113, 64) output shape: (None, 8, 113, 64)
p_re_lu_4: input shape: (None, 8, 113, 64) output shape: (None, 8, 113, 64)
reshape_1: input shape: (None, 8, 113, 64) output shape: (None, 8, 7232)
1_lstm_layer: input shape: (None, 8, 7232) output shape: (None, 8, 128)
2_dropout_layer: input shape: (None, 8, 128) output shape: (None, 8, 128)
2_lstm_layer: input shape: (None, 8, 128) output shape: (None

In [22]:
batchSize = 200
train_epoches = 50

model.fit(reshaped_train,train_labels,validation_data=(reshaped_val,val_labels),epochs=train_epoches,batch_size=batchSize,verbose=1)

print('Calculating score.. ')
score = model.evaluate(reshaped_test,test_labels,verbose=1)
print(score)

train_epoches = 20
all_train = np.concatenate((reshaped_train, reshaped_val))
all_labels = np.concatenate((train_labels, val_labels))
model.fit(all_train,all_labels,validation_data=(reshaped_test,test_labels),epochs=train_epoches,batch_size=batchSize,verbose=1)

print('Calculating score.. ')
score = model.evaluate(reshaped_test,test_labels,verbose=1)
print(score)
model.save('taskB2_all_Subjects_CNN_LSTM_our_elaboration_full.h5')

Train on 41416 samples, validate on 5078 samples
Epoch 1/50
41416/41416 [==============================] - 70s 2ms/step - loss: 1.1169 - acc: 0.7113 - val_loss: 0.6594 - val_acc: 0.8358
Epoch 2/50
41416/41416 [==============================] - 64s 2ms/step - loss: 0.7242 - acc: 0.7747 - val_loss: 0.5463 - val_acc: 0.8387
Epoch 3/50
41416/41416 [==============================] - 64s 2ms/step - loss: 0.5851 - acc: 0.7989 - val_loss: 0.5238 - val_acc: 0.8513
Epoch 4/50
41416/41416 [==============================] - 65s 2ms/step - loss: 0.4985 - acc: 0.8214 - val_loss: 0.4790 - val_acc: 0.8584
Epoch 5/50
41416/41416 [==============================] - 64s 2ms/step - loss: 0.4499 - acc: 0.8339 - val_loss: 0.4800 - val_acc: 0.8578
Epoch 6/50
41416/41416 [==============================] - 64s 2ms/step - loss: 0.4003 - acc: 0.8463 - val_loss: 0.4748 - val_acc: 0.8673
Epoch 7/50
41416/41416 [==============================] - 64s 2ms/step - loss: 0.3665 - acc: 0.8595 - val_loss: 0.4605 - val_acc:

Epoch 9/20
46494/46494 [==============================] - 74s 2ms/step - loss: 0.0368 - acc: 0.9894 - val_loss: 0.6037 - val_acc: 0.9097
Epoch 10/20
46494/46494 [==============================] - 74s 2ms/step - loss: 0.0378 - acc: 0.9892 - val_loss: 0.5483 - val_acc: 0.9086
Epoch 11/20
46494/46494 [==============================] - 74s 2ms/step - loss: 0.0421 - acc: 0.9873 - val_loss: 0.5597 - val_acc: 0.9094
Epoch 12/20
46494/46494 [==============================] - 74s 2ms/step - loss: 0.0347 - acc: 0.9898 - val_loss: 0.6034 - val_acc: 0.9106
Epoch 13/20
46494/46494 [==============================] - 74s 2ms/step - loss: 0.0341 - acc: 0.9902 - val_loss: 0.5999 - val_acc: 0.9095
Epoch 14/20
46494/46494 [==============================] - 74s 2ms/step - loss: 0.0339 - acc: 0.9901 - val_loss: 0.6719 - val_acc: 0.9087
Epoch 15/20
46494/46494 [==============================] - 74s 2ms/step - loss: 0.0352 - acc: 0.9900 - val_loss: 0.5919 - val_acc: 0.9081
Epoch 16/20
46494/46494 [==========

In [15]:
model = load_model('../taskB2_all_Subjects_CNN_LSTM_our_elaboration_full.h5')

Instructions for updating:
keep_dims is deprecated, use keepdims instead
Instructions for updating:
keep_dims is deprecated, use keepdims instead


In [16]:
class My_History(ke.callbacks.Callback):

    def on_train_begin(self, logs={}):
        self.f1_scores = []
        self.test_acc = []
        self.f1_scores_avg = []
        self.f1_scores_epoch = []

    def on_train_end(self, logs={}):
        return
    def on_epoch_begin(self, epoch, logs={}):
        return

    def on_epoch_end(self, epoch, logs={}):
        val_accuracy = logs.get('val_acc')
        print('Test accuracy is {}'.format(val_accuracy))
        self.test_acc.append(val_accuracy)
        class_predictions = []
        class_true = []
        predictions = self.model.predict(self.validation_data[0])
        for pair in zip(predictions, self.validation_data[1]):
            class_predictions.append(np.argmax(pair[0]))
            class_true.append(np.argmax(pair[1]))

        f1_scores_i = f1_score(class_predictions, class_true, average=None)
        self.f1_scores_epoch.append(f1_scores_i)
        self.f1_scores_avg.append(np.mean(f1_scores_i))
        self.f1_scores.append(f1_score(class_true, class_predictions, average='weighted'))

    def on_batch_begin(self, batch, logs={}):
        return
    def on_batch_end(self, batch, logs={}):
        return

In [19]:
print('Traning model again adding validation data...')
train_epochs = 30
batchSize = 200
test_filename = './test_phase_log.csv'
csv_logger_2 = CSVLogger(test_filename, separator=',', append=False)
my_callback = My_History()
checkpoint_2 = ModelCheckpoint('best_taskB2_all_Subjects_CNN_LSTM.h5', monitor='val_acc', verbose=1, save_best_only=True)
reduce_lr_2 = ReduceLROnPlateau(monitor='val_acc', factor=0.1, patience=5, mode='max', verbose=1, min_lr=0)
all_train = np.concatenate((reshaped_train, reshaped_val))
all_labels = np.concatenate((train_labels, val_labels))
model.fit(all_train,all_labels,validation_data=(reshaped_test,test_labels),epochs=train_epochs,batch_size=batchSize,callbacks=[reduce_lr_2, csv_logger_2, my_callback, checkpoint_2],verbose=1)

print('After other {0} epochs BEST test accuracy is: {1}, and BEST f1-score is {2}'.format(train_epochs, np.amax(my_callback.test_acc), np.amax(my_callback.f1_scores)))


Traning model again adding validation data...
Train on 46494 samples, validate on 9894 samples
Epoch 1/30
46400/46494 [============================>.] - ETA: 0s - loss: 0.0302 - acc: 0.9914Test accuracy is 0.9135839895742311
Epoch 00001: val_acc improved from -inf to 0.91358, saving model to best_taskB2_all_Subjects_CNN_LSTM.h5
46494/46494 [==============================] - 82s 2ms/step - loss: 0.0302 - acc: 0.9914 - val_loss: 0.5959 - val_acc: 0.9136
Epoch 2/30
46400/46494 [============================>.] - ETA: 0s - loss: 0.0242 - acc: 0.9930Test accuracy is 0.908227211589979
Epoch 00002: val_acc did not improve
46494/46494 [==============================] - 81s 2ms/step - loss: 0.0242 - acc: 0.9930 - val_loss: 0.6525 - val_acc: 0.9082
Epoch 3/30
46400/46494 [============================>.] - ETA: 0s - loss: 0.0238 - acc: 0.9930Test accuracy is 0.9055993528059572
Epoch 00003: val_acc did not improve
46494/46494 [==============================] - 80s 2ms/step - loss: 0.0238 - acc: 0.9

Epoch 27/30
46400/46494 [============================>.] - ETA: 0s - loss: 0.0025 - acc: 0.9994Test accuracy is 0.9176268459980912
Epoch 00027: val_acc did not improve
46494/46494 [==============================] - 79s 2ms/step - loss: 0.0025 - acc: 0.9994 - val_loss: 0.7623 - val_acc: 0.9176
Epoch 28/30
46400/46494 [============================>.] - ETA: 0s - loss: 0.0023 - acc: 0.9995Test accuracy is 0.9178289885180682
Epoch 00028: val_acc improved from 0.91783 to 0.91783, saving model to best_taskB2_all_Subjects_CNN_LSTM.h5
46494/46494 [==============================] - 79s 2ms/step - loss: 0.0023 - acc: 0.9995 - val_loss: 0.7649 - val_acc: 0.9178
Epoch 29/30
46400/46494 [============================>.] - ETA: 0s - loss: 0.0024 - acc: 0.9994
Epoch 00029: reducing learning rate to 1.0000000656873453e-06.
Test accuracy is 0.9175257735332384
Epoch 00029: val_acc did not improve
46494/46494 [==============================] - 79s 2ms/step - loss: 0.0025 - acc: 0.9994 - val_loss: 0.7657 -

NameError: name 'sys' is not defined

In [20]:
# saving variables
open_file = 'results_taskB2_all_Subjects_CNN_LSTM.pkl'
print('Saving results to: ' + open_file)
with open(open_file, 'wb') as f:
    pk.dump([my_callback.test_acc, my_callback.f1_scores, my_callback.f1_scores_avg, my_callback.f1_scores_epoch], f)

Saving results to: results_taskB2_all_Subjects_CNN_LSTM.pkl


In [21]:
best_model = load_model('best_taskB2_all_Subjects_CNN_LSTM.h5')
predictions = best_model.predict(reshaped_test)

In [40]:
# F1-score measure
from sklearn.metrics import f1_score
num_classes = 18
class_predictions = []
class_true = []
tot_labels = 0.0
count = 0.0
for pair in zip(predictions, test_labels):
    class_predictions.append(np.argmax(pair[0]))
    class_true.append(np.argmax(pair[1]))
    if np.argmax(pair[0]) == np.argmax(pair[1]):
        count += 1.0
    tot_labels += 1.0
    
print('Standard accuracy is ' + str(count/tot_labels))    

unique, counts = np.unique(class_true, return_counts=True)
counted_labels = dict(zip(unique, counts))
f1_scores = f1_score(class_predictions, class_true, average=None)

tot_f1_score = 0.0
weights_sum = 0.0
for i in range(num_classes):
    labels_class_i = counted_labels[i]
    weight_i = labels_class_i / tot_labels
    weights_sum += weight_i
    tot_f1_score += f1_scores[i]*weight_i
    print(str(i) + ' ' + str(weight_i) + ' ' + str(f1_scores[i]))

    
print('The weigths sum is ' + str(weights_sum))
print('The computed f1-score is {}'.format(tot_f1_score))
print('The f1-score with sklearn function is {}'.format(f1_score(class_true, class_predictions, average='weighted')))
print('Average f1-score is {}'.format(np.mean(f1_scores)))

Standard accuracy is 0.9162118455629674
0 0.832524762482 0.958662832495
1 0.0058621386699 0.704918032787
2 0.00960177885587 0.896174863388
3 0.00606428138266 0.730769230769
4 0.00838892257934 0.898203592814
5 0.0230442692541 0.703448275862
6 0.0161714170204 0.801324503311
7 0.0101071356378 0.532710280374
8 0.00778249444108 0.554913294798
9 0.00394178289873 0.506024096386
10 0.00424499696786 0.463768115942
11 0.0040428542551 0.514285714286
12 0.00262785526582 0.464285714286
13 0.0067717808773 0.69918699187
14 0.00616535273903 0.655172413793
15 0.0100060642814 0.634146341463
16 0.0320396199717 0.721723518851
17 0.0106124924196 0.569620253165
The weigths sum is 1.0
The computed f1-score is 0.9132369328608785
The f1-score with sklearn function is 0.9132369328608785
Average f1-score is 0.6671854481466055


In [ ]:
from keras.models import Model
from sklearn.decomposition import PCA

layer_name = 'dense_layer'
inter_layer_model = Model(inputs=model.input, outputs=model.get_layer(layer_name).output)

network_output = inter_layer_model.predict(all_train)

# PCA with certain number of principal components
pca = PCA(n_components=40)
pca.fit(network_output)
new_output = pca.transform(network_output)

In [ ]:
from sklearn import svm
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import cross_val_predict

svm_labels = np.argmax(all_labels, axis=1)

# train a one-vs-one multi-class support vector machine
clf = svm.SVC(decision_function_shape='ovo')
clf.fit(new_output, svm_labels)

# predict test data
svm_pred = clf.predict(pca.transform(inter_layer_model.predict(reshaped_test)))

# measure accuracy and f1-score
num = 0.0
den = 0.0
new_test_labels = np.argmax(test_labels, axis=1)
for pair in zip(svm_pred, new_test_labels):
    if pair[0] == pair[1]:
        num += 1.0
    
    den += 1.0

scores = cross_val_score(clf, pca.transform(inter_layer_model.predict(reshaped_test)),new_test_labels, cv=5)
print(scores)
print("Accuracy: %0.2f (+/- %0.2f)" % (scores.mean(), scores.std() * 2))

print('Test accuracy is: {}'.format(num / den))
f1_scores = f1_score(svm_pred, new_test_labels, average=None)
print('The f1-score with sklearn function is {}'.format(f1_score(new_test_labels, svm_pred, average='weighted')))
print('Average f1-score is {}'.format(np.mean(f1_scores)))

In [ ]:
from sklearn.model_selection import cross_val_predict

predicted = cross_val_predict(clf, pca.transform(inter_layer_model.predict(reshaped_test)),new_test_labels, cv=5)

# measure accuracy and f1-score
num = 0.0
den = 0.0
new_test_labels = np.argmax(test_labels, axis=1)
for pair in zip(predicted, new_test_labels):
    if pair[0] == pair[1]:
        num += 1.0
    
    den += 1.0

print('Test accuracy is: {}'.format(num / den))
f1_scores = f1_score(predicted, new_test_labels, average=None)
print('The f1-score with sklearn function is {}'.format(f1_score(new_test_labels, predicted, average='weighted')))
print('Average f1-score is {}'.format(np.mean(f1_scores)))

In [ ]:
pred_df = pd.DataFrame(predictions)
pred_df.to_csv('preds_test.csv', header=False, index=False)

In [ ]:
true_df = pd.DataFrame(testY)
true_df.to_csv('true_test.csv', header=False, index=False)